In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings

# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../..'))
warnings.filterwarnings("ignore", category=FutureWarning)


# Import MCP components
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# Set up the model

#model = ChatOpenAI(model="gpt-4o")
from dotenv import load_dotenv
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GEMINI_API_KEY"))


In [2]:
server_params = StdioServerParameters(
    command="python",
    #Replace with the full path to the tickertick.py file
    args=["/Users/chen/Library/Mobile Documents/com~apple~CloudDocs/NYU/SPRING 25/TECH-UB 24/StocksFlags/src/mcp_server/tickertick.py"],
)

# Test connection to the server
async def test_server(query):
    print("Testing connection to Tickertick MCP server...")
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            print("Connected to Tickertick MCP server successfully!")
            
            # List available tools
            tools = await load_mcp_tools(session)
            print(f"Found {len(tools)} tools:")
            for tool in tools:
                print(f"- {tool.name}: {tool.description}")
            
            # Test a tool with LangChain
            agent = create_react_agent(model, tools)
            print(f"\nTesting agent with query: '{query}'")
            agent_response = await agent.ainvoke({"messages": query})
            return agent_response



In [3]:
query = "what's the trending news on bloomberg?"

In [4]:
response = await test_server(query)

Testing connection to Tickertick MCP server...
Connected to Tickertick MCP server successfully!
Found 7 tools:
- get_ticker_news_tool: 
    Get news for a specific ticker symbol.
    
    Args:
        ticker: The ticker symbol (e.g., AAPL, MSFT, TSLA)
        limit: Maximum number of news items to return (default: 30, max: 100)
        
    Returns:
        A dictionary containing news items related to the ticker
    
- get_broad_ticker_news_tool: 
    Get broader news for a specific ticker symbol.
    
    Args:
        ticker: The ticker symbol (e.g., AAPL, MSFT, TSLA)
        limit: Maximum number of news items to return (default: 30, max: 100)
        
    Returns:
        A dictionary containing broader news items related to the ticker
    
- get_news_from_source_tool: 
    Get news from a specific source.
    
    Args:
        source: The news source (e.g., bloomberg, wsj, cnbc)
        limit: Maximum number of news items to return (default: 30, max: 100)
        
    Returns:


In [5]:
print(type(response))
print(response['messages'][-1].content)

<class 'langgraph.pregel.io.AddableValuesDict'>
Here's a summary of the trending news from Bloomberg:

*   **Trade and Tariffs:** Several articles discuss the impact of trade tensions and tariffs, including their effects on Canada's energy security, US businesses, and African nations. There's also news about "progress" in US-Japan trade talks.
*   **Central Banks and Interest Rates:** News includes Turkey's surprise rate hike, Ukraine holding its key interest rate, and discussions between the IMF and Bangladesh.
*   **Company News:** Several companies are in the news, including BP, Ally Financial, Charles Schwab, Eli Lilly, Infosys, American Express, Global Payments, and Blackstone. Eli Lilly is soaring after weight-loss pill works as well as Ozempic shot
*   **Economic Concerns:** Articles discuss the possibility of a recession and the impact of pre-tariff shopping on the US GDP.
*   **Technology and AI:** There are stories about AWS's view on the future of contact centers with AI, an